In [6]:
import requests,json
from bs4 import BeautifulSoup

In [3]:
def voting_list(state,district,vdc_mun,ward_no):
    ''' 
        This script/module scrapes data from nepal government 
        website:"https://election.gov.np/np/page/voter-list-db" 
        ,parse the data and save it to a json file.
        -----------------------------------------------
        eg:
        state = {'name':'Provision_1','no':1}
        district = {'name':'Sunsari','no':10}
        vcd_mun = {'name':'ईटहरी उपमहानगरपालिका','no':5092}  
            // vdc/mun name appears in the data so for better provide in nepali
        ward_no = [2]
            // can provide lists of ward

        \\Note: state no, ward no, district no & vcd_mun no can be found on the original site html file 

        >>>> Itahari_data = voting_list(state,district,vdc_mun,ward_no)  //Returns a list of dictionaries
        >>>> save('itahari_data',Itahari_data) // It save data to itahari_data.json file 
        
        OR ``can scrape multiple munipalities and save to a single file``
        
        >>>> kathmandu_metro_data = voting_list(state,district,vdc_mun,ward_no)
        >>>> save('Itahari_&_kathmandu_data',kathmandu_metro_data,Itahari_data)
             // It saves both itahari and ktm data  
    '''
    state_name = state['name']
    state_no = state['no']
    district_name = district['name']
    district_no = district['no']
    vdc_mun_name = vdc_mun['name']
    vdc_mun_no = vdc_mun['no']
    list_data = []
    for ward in ward_no:
        payload = {
            'state':f'{state_no}',
            'district':f'{district_no}',
            'vdc_mun':f'{vdc_mun_no}',
            'ward':f'{ward}',
            'reg_centre':''
                }
        url = 'https://voterlist.election.gov.np/bbvrs/view_ward_1.php'
        response = requests.post(url,data=payload)
        soup = BeautifulSoup(response.text,"html.parser") 
        div = soup.find('div',class_='div_bbvrs_data')
        table = div.find("table")
        head_table_data = ['voter_id','voter_name','age','gender','spouse_name','parent_name','vdc_mun','ward']
        body_table_data = table.tbody.find_all("tr")  
        for count in range(len(body_table_data)):
            dict_data = {}
            for i,th in enumerate(head_table_data):
                if i <=5:
                    dict_data[th] = body_table_data[count].find_all("td")[i+1].get_text()   
                elif i == 6:
                    dict_data[th] = vdc_mun_name
                else:
                    dict_data[th] = str(ward)
            list_data.append(dict_data)
    numbers_map = {'0': '०','1': '१','2': '२','3': '३','4': '४','5': '५','6': '६','7': '७','8': '८','9': '९'}
    num_unicode_ls = ['voter_id','age','ward']
    for data in list_data:
        for key in data.keys():
            if key in num_unicode_ls:
                for eng_num,unicode  in numbers_map.items():
                    data[key] = data[key].replace(eng_num,unicode)
    return list_data 

def save(file_name,list_,*args):   
    for arg in args:
        list_ += arg
    with open(f'{file_name}.json','w',encoding='utf8') as file:
        json.dump(list_,file,ensure_ascii=False)

In [4]:
ktm_ward = [2, 3, 4, 5]
budhanilkantha_ward = [3, 4, 5, 6, 7 ]
tokha_mun_ward = [2, 3, 4, 5, 6, 7]
state = {"name" : "बागमती प्रदेश",
       "no" : "3"}
district = {"name" : "काठमाण्डौ",
            "no" : "26"}
ktm = {"name" : "काठमाण्डौ महानगरपालिका",
            "no" : "5278"}
budhanilkantha = {"name" : "बुढानिलकण्ठ नगरपालिका",
                 "no" : "5286"}
tokha_mun = {"name" : "टोखा नगरपालिका",
            "no" : "5282"}

In [ ]:
ktm_metro = voting_list(state,district,ktm,ktm_ward)
budhanilkantha_metro = voting_list(state,district,budhanilkantha,budhanilkantha_ward)
tokha_metro = voting_list(state,district,tokha_mun,tokha_mun_ward)

In [24]:
save('final_voting_list',ktm_metro,budhanilkantha_metro,tokha_metro)